# [lenet03] BaseUpdater As an Updater Template

在這個教學裡面，會告訴你怎麼透過模板，調整訓練中的反向傳播與網路更新的流程。

In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

## Introduction

In [4]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from modules.base.updater import BaseUpdater
from print_source import print_source

print_source(BaseUpdater)

Updater (更新器) 的用法很簡單：
1. 登記 `module` 得到 module updater function: `module_update = updater(module)`，這個用法的實現可以參考類別方法 `register_module` (`__call__`是語法糖設計)。
2. 更新網路權重使用 `module_update`: 在 trainer 裡面透過 `loss = module_update(image, target)` 更新網路權重並回傳 loss

如果需要改寫的話，只需要重新實現 `update` 這個方法就可以了，切記參數中要包含 `module` 用來登記在。

P.S: 做的嚴謹一點，也可以順便實現 `check_module` 這個方法，確保 `update` 中所使用的元件都有在 `module` 裡面，例如在原先的實現中我們需要optimizer和critterion。<br>
&emsp;&thinsp;&thinsp;&thinsp;&thinsp;這樣做的好處是，一旦你魔改了原有的 `module` 但仍想沿用原先的 `updater`，會提醒你哪些元件需要被實現。


## Example

SGD 和 Cosine Scheduler 是一個很常見的組合。<br>
這邊我們示範怎麼透過繼承 BaseUpdater 的方式，實現一個 CustomUpdater 來強迫模型使用 Cosine Scheduler 和 SGD 更新梯度。

原則上**不**鼓勵這樣設計，因為把神經網路的元件寫在更新器裡面不太直覺、也因此不容易被找到。<br>
但這邊僅僅作為釋例提供一種可能的做法。

In [3]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import torch
from functools import partial
from modules.base.updater import BaseUpdater

class CustomUpdater(BaseUpdater):
    """Base class of updaters."""

    def __init__(self, criterion=None):
        self.criterion = criterion

    def register_module(self, module):
        self.check_module(module)
        # --- Modified 
        optimizer = torch.optim.SGD(module.net.parameters(), lr=1)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0, last_epoch=-1)
        module.optimizer = optimizer
        module.scheduler = scheduler
        # --- Modified
        return partial(self.update, module)

    def update(self, module, images, targets, **kwargs) -> float:
        module.optimizer.zero_grad()
        preds = module(images)
        loss = module.criterion(preds, targets)
        loss.backward()
        module.optimizer.step()
        module.scheduler.step() # >> Modified
        return loss.item()

我們在 `register_module` 中把 `module` 改成 SGD 並且加入 scheduler。<br>
在 `update` 中配合著修改更新梯度的流程。

In [5]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from modules.base.trainer import BaseTrainer, TrainLogger
from modules.base.updater import BaseUpdater
from modules.base.validator import BaseValidator

from torch import nn
from mnist_dataloaders import train_dataloader, val_dataloader, test_dataloader
from lenet import LeNet5,  batch_acc

validator = BaseValidator(metric=batch_acc)
updater = CustomUpdater()
trainer = BaseTrainer(max_iter=4000, eval_step=4000, metric=batch_acc)

# train
print("Train:")
lenet = LeNet5().cuda()
trainer.train(module=lenet, updater=updater, train_dataloader=train_dataloader, val_dataloader=val_dataloader)

# test
print("\n Test:")
lenet.load("./checkpoints")
validator.validation(module=lenet, dataloader=test_dataloader)

Train:
--------
Device: cuda
# of Training Samples: 211
# of Validation Samples: 47
Max iteration: 4000 steps (validates per 4000 steps)
Checkpoint directory: ./checkpoints/
Evaluation metric: function
--------


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

2024-08-19 15:18:46.251 | SUCCESS  | modules.base.trainer:success:71 - Model saved! Validation: (New) 0.11113 > (Old) 0.00000

 Test:


  0%|          | 0/79 [00:00<?, ?it/s]

0.11224287974683544

你會發現沒有辦法訓練好了！ <br>

嗯...事實上這邊完整復刻了 LeCun(1998) 的網路架構 (沒偷用 Max Pooling 和 ReLU)，想用 SGD 訓練好還是有點不容易的。<br>
原因出在適合 Sigmoid activation 的權重初始化方法是 Xavier initialization，<br>
而 PyTorch 預設的權重初始化是針對 ReLU 的 He Initialzation。 <br>
可以參考：[Deep Learning Wizard - Weight Initializations & Activation Functions](https://www.deeplearningwizard.com/deep_learning/boosting_models_pytorch/weight_initialization_activation_functions/)

讓我們將網路權重透過 Xavier 初始化後再做一遍。

In [6]:
print("Train:")
lenet = LeNet5().cuda()
# torch.nn.init.xavier_uniform_(lenet.net.weight)
for layer in lenet.net.modules():
    if isinstance(layer, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_uniform_(layer.weight)

trainer.train(module=lenet, updater=updater, train_dataloader=train_dataloader, val_dataloader=val_dataloader)

# test
print("\n Test:")
lenet.load("./checkpoints")
validator.validation(module=lenet, dataloader=test_dataloader)

Train:
--------
Device: cuda
# of Training Samples: 211
# of Validation Samples: 47
Max iteration: 4000 steps (validates per 4000 steps)
Checkpoint directory: ./checkpoints/
Evaluation metric: function
--------


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

2024-08-19 15:22:47.741 | SUCCESS  | modules.base.trainer:success:71 - Model saved! Validation: (New) 0.96182 > (Old) 0.00000

 Test:


  0%|          | 0/79 [00:00<?, ?it/s]

0.9678599683544303